# Get to Start

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Clustering_EM_Heuristic/Joint-Xiaoyu/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-learn seaborn mlflow omegaconf
from mlflow import log_metric, log_text, log_param
from omegaconf import DictConfig, ListConfig
from sklearn.metrics import f1_score
from scipy.io import arff
!pip install gurobipy
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd
from copy import deepcopy
from pyomo.environ import *
import time
import sys
import os
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 47.9 MB/s eta 0:00:00


In [ ]:
from inputlds import *

In [ ]:
!pip install mlflow omegaconf
from mlflow import log_metric, log_text, log_param
from omegaconf import DictConfig, ListConfig

# Functions

## Methods

*   IF-Gurobi
*   IF
*   EM
*   FFT
*   DTW



In [ ]:
import os
import sys
import time
import math
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from pyomo.environ import *

class MIP_IF():

    """Estimator based on NCPOP Regressor

    References
    ----------
    Quan Zhou https://github.com/Quan-Zhou/Proper-Learning-of-LDS/blob/master/ncpop/functions.py

    """

    def ind_Gurobi_function(self, X, label, UB0, UB1, M, T, reg):
        from scipy.io import arff
        !pip install gurobipy
        import gurobipy as gp
        # from gurobipy import *

        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals."""

        e = gp.Env()
        e.setParam('TimeLimit', 6*60)
        model = gp.Model(env=e)
        T -= 1
        I = len(X)
        print(UB0,UB1,T, M)

        # Var
        L = model.addVars(I, name="L", vtype='B')  # Indicator

        f0 = model.addVars(M, T, name="f0", vtype='C')
        phi0 = model.addVars(UB0, (T + 1), name="phi0", vtype='C')
        G0 = model.addVars(UB0, UB0, name="G0", vtype='C')
        F0 = model.addVars(M, UB0, name="F0", vtype='C')
        q0 = model.addVars(UB0, T, name="q0", vtype='C')
        p0 = model.addVars(M, T, name="p0", vtype='C')
        quatr0 = model.addVars(M, (T + 1), name="quatr0", vtype='C')
        quatr_hidden0 = model.addVars(UB0, T, name="quatr_hidden0", vtype='C')

        f1 = model.addVars(M, T, name="f1", vtype='C')
        phi1 = model.addVars(UB1, (T + 1), name="phi1", vtype='C')
        G1 = model.addVars(UB1, UB1, name="G1", vtype='C')
        F1 = model.addVars(M, UB1, name="F1", vtype='C')
        q1 = model.addVars(UB1, T, name="q1", vtype='C')
        p1 = model.addVars(M, T, name="p1", vtype='C')
        quatr1 = model.addVars(M, (T + 1), name="quatr1", vtype='C')
        quatr_hidden1 = model.addVars(UB1, T, name="quatr_hidden1", vtype='C')
        z0_squared = model.addVars(I, T, M, name="z0_squared", vtype='C')
        z1_squared = model.addVars(I, T, M, name="z1_squared", vtype='C')


        # Obj
        obj = gp.quicksum(
            (1 - L[i]) * z0_squared[i, t, m] + L[i] * z1_squared[i, t, m]
            for m in range(M) for t in range(T) for i in range(I)
        )
        obj += gp.quicksum(reg * q0[n0, t] ** 2 for n0 in range(UB0) for t in range(T))
        obj += gp.quicksum(reg * q1[n1, t] ** 2 for n1 in range(UB1) for t in range(T))
        obj += gp.quicksum(reg * p0[m, t] ** 2 + reg * p1[m, t] ** 2 for m in range(M) for t in range(T))

        model.setObjective(obj, GRB.MINIMIZE)

        # Constraint
        model.addConstrs(z0_squared[i, t, m] == (X[i, t, m] - f0[m, t])**2 for i in range(I) for t in range(T) for m in range(M))
        model.addConstrs(z1_squared[i, t, m] == (X[i, t, m] - f1[m, t])**2 for i in range(I) for t in range(T) for m in range(M))

        for t in range(T):
            for m in range(M):
                model.addConstr(gp.quicksum(F0[m, n] * phi0[n, t+1] for n in range(UB0)) == quatr0[m, t+1])
                model.addConstr(gp.quicksum(F1[m, n] * phi1[n, t+1] for n in range(UB1)) == quatr1[m, t+1])
        model.addConstr(gp.quicksum(f0[m, t] - quatr0[m,(t+1)] - p0[m, t]for t in range(T) for m in range(M)) == 0)
        model.addConstr(gp.quicksum(f1[m, t] - quatr1[m,(t+1)] - p1[m, t]for t in range(T) for m in range(M)) == 0)
        for t in range(T):
            for n in range(UB0):
                model.addConstr(gp.quicksum(G0[n, nn] * phi0[nn, t] for nn in range(UB0)) == quatr_hidden0[n, t])
            for n in range(UB1):
                model.addConstr(gp.quicksum(G1[n, nn] * phi1[nn, t] for nn in range(UB1)) == quatr_hidden1[n, t])
        model.addConstr(gp.quicksum(phi0[n, t+1] - quatr_hidden0[n, t]- q0[n, t] for t in range(T) for n in range(UB0))  == 0)
        model.addConstr(gp.quicksum(phi1[n, t+1] - quatr_hidden1[n, t]- q1[n, t] for t in range(T) for n in range(UB1))  == 0)

        model.update()
        model.Params.NonConvex = 2

        model.optimize()

        if model.status == GRB.Status.OPTIMAL:
            print("THIS IS OPTIMAL SOLUTION")
        else:
            print("THIS IS NOT OPTIMAL SOLUTION")

        data_dict = {
            'G0': [model.getAttr("x", G0)[h, k] for (h, k) in model.getAttr("x", G0)],
            'F0': [model.getAttr("x", F0)[h, k] for (h, k) in model.getAttr("x", F0)],
            'G1': [model.getAttr("x", G1)[h, k] for (h, k) in model.getAttr("x", G1)],
            'F1': [model.getAttr("x", F1)[h, k] for (h, k) in model.getAttr("x", F1)]
        }

        label_out = np.array([round(model.getAttr("x", L)[h]) for h in model.getAttr("x", L)])

        # print(f'M={M}, T={T}')
        return model, label_out, data_dict

    def system_matrix(self, arr_G, arr_F, M, UB):
        arr_rounded_G = np.round(arr_G, decimals=3)
        arr_rounded_F = np.round(arr_F, decimals=3)
        arr_rounded_G[np.abs(arr_rounded_G) < 0.001] = 0
        arr_rounded_F[np.abs(arr_rounded_F) < 0.001] = 0
        rows_G, cols_G = arr_G.shape
        rows_F, cols_F = arr_F.shape

        if not arr_rounded_G.any() and not arr_rounded_F.any():
            print("Warning: arr_rounded_G is empty or contains only zeros.")
            return arr_G, arr_F,0

        else:
            nonzero_rows_G, nonzero_cols_G = np.nonzero(arr_rounded_G.reshape(UB, UB))
            nonzero_rows_F, nonzero_cols_F = np.nonzero(arr_rounded_F.reshape(M,UB))
            def max_max(G):
                if G.size ==0:
                    return 0
                else:
                    return max(G)
            N = max(max_max(nonzero_rows_G), max_max(nonzero_cols_G), max_max(nonzero_cols_F)) + 1
            # print(f"N:{N}")
            system_G = arr_G[:N, :N]
            system_F = arr_F[:, :N]
            return system_G, system_F, N

    def obj_function(self, model):
        obj_1 = sum((1-model.L[i])*(model.X[i][t,m]-model.f[m,t])**2 + model.L[i]*(model.X[i][t,m]-model.ff[m,t])**2 for i in model.I for m in model.M for t in model.T1)
        obj_2 = reg*sum(model.p[m,t]**2 for m in model.M for t in model.T1) + reg*sum(model.q[n,t]**2 for n in model.N for t in model.T1)
        obj_3 = reg*sum(model.pp[m,t]**2 for m in model.M for t in model.T1) + reg*sum(model.qq[n,t]**2 for n in model.N for t in model.T1)
        return obj_1+obj_2+obj_3

    def ind_Bonmin_function(self, X,label,N,M,T=20,option='bonmin', reg=270):

        I = len(X)
        # T = len(X[0])
        # print(I,T)

        #---------------------------------------------#
        # Dimension of G & F  - G:[N,N]  Fdash:[M,N]  #
        #---------------------------------------------#
        # N = 2
        # M = 1

        # Create model
        model = ConcreteModel()

        # Create index set
        model.N = RangeSet(0, N-1)
        model.M = RangeSet(0, M-1)
        model.I = RangeSet(0, I-1)
        model.T1 = RangeSet(0,T-1)
        model.T2 = RangeSet(0,T)

        # Create variables
        model.L = Var(model.I, within=NonNegativeIntegers, bounds=(0,1), initialize =label) #indicator

        model.f = Var(model.M, model.T1, domain=Reals)
        model.m = Var(model.N, model.T2, domain=Reals)
        model.G = Var(model.N, model.N, domain=Reals)
        model.Fdash = Var(model.M, model.N, domain=Reals)
        model.q = Var(model.N, model.T1, domain=Reals)
        model.p = Var(model.M, model.T1, domain=Reals)

        model.ff = Var(model.M, model.T1, domain=Reals)
        model.mm = Var(model.N, model.T2, domain=Reals)
        model.GG = Var(model.N, model.N, domain=Reals)
        model.FFdash = Var(model.M, model.N, domain=Reals)
        model.qq = Var(model.N, model.T1, domain=Reals)
        model.pp = Var(model.M, model.T1, domain=Reals)

        # Create parameters
        model.X = Param(model.I, initialize = X,mutable=False)

        # Create objective function
        model.OBJ = Objective(rule=self.obj_function, sense=minimize)

        # Create constrains
        model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
        model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
        model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
        model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)

        model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
        model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
        model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
        model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)

        solver = SolverFactory(option)
        solver.solve(model, tee=True)
        # solver.solve(model, tee=True).write()

        # solver = SolverManagerFactory('neos')
        # result = solver.solve(model, opt='mosek')

        # model.display()
        iid = []
        for i in model.L:
          iid.append(model.L[i].value)

        return model, np.array(iid)

    def obj_function_2(self, model):
        obj_1 = sum((model.K[k][t,m]-model.f[m,t])**2 for k in model.KK for m in model.M for t in model.T1)
        obj_2 = sum((model.L[l][t][m]-model.ff[m,t])**2 for l in model.LL for m in model.M for t in model.T1)
        obj_3 = reg*sum(model.p[m,t]**2 for m in model.M for t in model.T1) + reg*sum(model.q[n,t]**2 for n in model.N for t in model.T1)
        obj_4 = reg*sum(model.pp[m,t]**2 for m in model.M for t in model.T1) + reg*sum(model.qq[n,t]**2 for n in model.N for t in model.T1)
        return obj_1+obj_2+obj_3+obj_4

    def SimCom(self, K,L,T,M,N, reg,option='bonmin'): # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
        """option: 'bonmin', 'cplex', 'mosek'..."""

        KK = len(K)
        LL = len(L)
        # T = 5
        print('cluster length:',KK,LL)

        # Create model
        model = ConcreteModel()
        # # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
        # M = 2
        # N = 2
        # Create index set
        model.M = RangeSet(0, M-1)
        model.N = RangeSet(0, N-1)
        model.KK = RangeSet(0,KK-1)
        model.LL = RangeSet(0,LL-1)
        model.T1 = RangeSet(0,T-1)
        model.T2 = RangeSet(0,T)

        # Create variables
        model.f = Var(model.M, model.T1, domain=Reals)
        model.m = Var(model.N, model.T2, domain=Reals)
        model.G = Var(model.N, model.N, domain=Reals)
        model.Fdash = Var(model.M, model.N, domain=Reals)
        model.q = Var(model.N, model.T1, domain=Reals)
        model.p = Var(model.M, model.T1, domain=Reals)

        model.ff = Var(model.M, model.T1, domain=Reals)
        model.mm = Var(model.N, model.T2, domain=Reals)
        model.GG = Var(model.N, model.N, domain=Reals)
        model.FFdash = Var(model.M, model.N, domain=Reals)
        model.qq = Var(model.N, model.T1, domain=Reals)
        model.pp = Var(model.M, model.T1, domain=Reals)

        # Create parameters
        model.K = Param(model.KK,initialize = K,mutable=False)
        model.L = Param(model.LL,initialize = L,mutable=False)

        # Create objective function

        model.OBJ = Objective(rule=self.obj_function_2, sense=minimize)

        # Create constrains
        model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
        model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
        model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
        model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)

        model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
        model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
        model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
        model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)

        solver = SolverFactory(option)
        solver.solve(model)
        # solver.solve(model, tee=True).write()

        # solver = SolverManagerFactory('neos')
        # result = solver.solve(model, opt='mosek')


        # model.pprint()
        return model

    #######################  CONCRETE MODEL  #######################
    # model with G constrains
    def SimCom_norm(self, K,L,T,M,N,reg,option='bonmin'): # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
        KK = len(K)
        LL = len(L)
        # T = 5
        print('cluster length:',KK,LL)

        # Create model
        model = ConcreteModel()

        model.M = RangeSet(0, M-1)
        model.N = RangeSet(0, N-1)
        model.KK = RangeSet(0,KK-1)
        model.LL = RangeSet(0,LL-1)
        model.T1 = RangeSet(0,T-1)
        model.T2 = RangeSet(0,T)

        # Create variables
        model.f = Var(model.M, model.T1, domain=Reals)
        model.m = Var(model.N, model.T2, domain=Reals)
        model.G = Var(model.N, model.N, domain=Reals)
        model.Fdash = Var(model.M, model.N, domain=Reals)
        model.q = Var(model.N, model.T1, domain=Reals)
        model.p = Var(model.M, model.T1, domain=Reals)

        model.ff = Var(model.M, model.T1, domain=Reals)
        model.mm = Var(model.N, model.T2, domain=Reals)
        model.GG = Var(model.N, model.N, domain=Reals)
        model.FFdash = Var(model.M, model.N, domain=Reals)
        model.qq = Var(model.N, model.T1, domain=Reals)
        model.pp = Var(model.M, model.T1, domain=Reals)

        # Create parameters
        model.K = Param(model.KK,initialize = K,mutable=False)
        model.L = Param(model.LL,initialize = L,mutable=False)

        # Create objective function

        model.OBJ = Objective(rule=self.obj_function_2, sense=minimize)

        # Create constrains
        model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
        model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
        model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
        model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)
        model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
        model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
        model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
        model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)
        model.con9 = Constraint(expr=sum(model.G[i,j] for i in model.N for j in model.N)-1==0)
        model.con10 = Constraint(expr=sum(model.GG[i,j] for i in model.N for j in model.N)-1==0)

        solver = SolverFactory(option)
        solver.solve(model)


        # model.pprint()
        return model


    def FFT_estimate(self, data, label, seed=42, is_plot=False):
        import numpy as np
        import pandas as pd
        import math
        import matplotlib.pyplot as plt
        from sklearn.metrics import f1_score
        from sklearn import preprocessing
        from sklearn.cluster import KMeans
        from sklearn.cluster import SpectralClustering
        import time

        X_train = data
        seed = seed
        np.random.seed(seed)
        index = np.arange(len(X_train))
        np.random.shuffle(index)

        X_train = X_train[index]
        X_label = label[index]
        print(X_label)

        data_fft = np.fft.rfft(X_train, axis=1)
        fft_features = np.abs(data_fft)
        feature_vectors = fft_features.reshape(len(X_train), -1)
        kmeans = KMeans(n_clusters=2, verbose=is_plot, random_state=seed)
        kmeans.fit(feature_vectors)
        y_pred = kmeans.labels_
        print(y_pred)
        '''
        if is_plot:
            for yi in range(2):
                plt.subplot(1, 2, 1+yi)
                for xx in X_train[y_pred == yi]:
                    if len(xx.shape) == 1:
                        plt.plot(xx, alpha=.2)
                    else:
                        plt.plot(xx[:,0], alpha=.2)
                        plt.plot(xx[:,1], alpha=.2)
                # plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
                # plt.xlim(0, 20)
                plt.ylim(-2, 4)
                plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
                        transform=plt.gca().transAxes)
                if yi == 1:
                    plt.title("FFT $k$-means")
            plt.tight_layout()
            plt.show()

            for freq in fft_features[y_pred==0]:
                if len(freq.shape) == 1:
                    plt.plot(freq, alpha=.5)
                else:
                    plt.plot(freq[:,0], alpha=.5)
                    plt.plot(freq[:,1], alpha=.5)
            plt.show()
            for freq in fft_features[y_pred==1]:
                if len(freq.shape) == 1:
                    plt.plot(freq, alpha=.5)
                else:
                    plt.plot(freq[:,0], alpha=.5)
                    plt.plot(freq[:,1], alpha=.5)
            plt.show()
        '''

        f1_1 = f1_score(X_label, y_pred)
        f1_2 = f1_score(X_label, 1-y_pred)
        if f1_1>f1_2:
            f1 = f1_1
        else:
            f1 = f1_2
        print(f1)
        return f1

    def DTW_estimate(self, data, label, seed=42, is_plot=False):
        import random
        from scipy.io import arff
        import sys
        sys.path.append('../')
        #from inputlds import *
        from tslearn.clustering import TimeSeriesKMeans
        from tslearn.datasets import CachedDatasets
        from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

        X_train = data
        X_label = label
        seed = seed
        np.random.seed(seed)
        index = np.arange(len(X_train))
        np.random.shuffle(index)
        # print(X_train[0])

        X_train = X_train[index]
        X_label = X_label[index]
        print(X_label)

        X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)
        sz = X_train.shape[1]
        print(sz)

        # Soft-DTW-k-means
        print("Soft-DTW k-means")
        sdtw_km = TimeSeriesKMeans(n_clusters=2,
                                max_iter=10,
                                metric="softdtw",
                                metric_params={"gamma": .01},
                                verbose=True,
                                random_state=seed)
        y_pred = sdtw_km.fit_predict(X_train)
        print(y_pred)
        '''
        if is_plot:
            for yi in range(2):
                plt.subplot(1, 2, 1+yi)
                for xx in X_train[y_pred == yi]:
                    plt.plot(xx.ravel(), "k-", alpha=.2)
                plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
                plt.xlim(0, sz)
                plt.ylim(-2, 4)
                plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
                        transform=plt.gca().transAxes)
                if yi == 1:
                    plt.title("Soft-DTW $k$-means")
            plt.tight_layout()
            plt.show()
        '''
        f1_1 = f1_score(X_label, y_pred)
        f1_2 = f1_score(X_label, 1-y_pred)
        if f1_1>f1_2:
            f1 = f1_1
        else:
            f1 = f1_2

        print('f1:',f1)
        return f1



    def MIP_estimate(self, data_in,label_in,method,N,name,reg, seed, MTS=True,option='bonmin',norm=True, thresh=0.75):
        from sklearn.metrics import f1_score

        """
        X_data.shape: N,T,M, for ecg, X_data contains all the data
        X_lebel.shape: N     for both, X_label is like [1,1,1,0,0,0]
        name: 'lds' or 'ecg'
        """

        M = data_in.shape[2]
        T = data_in.shape[1]
        print("m,t:",M,T)

        np.random.seed(seed)
        idx = np.arange(len(data_in))
        np.random.shuffle(idx)
        label = label_in[idx]
        #print("lds initialized label:", label)
        X = data_in[idx, :, : ]
        if method =='IF':
            model, label_out = self.ind_Bonmin_function(X=data_in, label=label_in, N=N,M=M,T=T,option='bonmin',reg =reg)
        elif method == 'IF-Gurobi':
            UB0,UB1=N,N
            model, label_out, data_dict = self.ind_Gurobi_function(X=data_in,label=label,UB0=UB0,UB1=UB1,M=M,T=T, reg =reg)
            G0 = np.array(np.array(data_dict['G0']).reshape(UB0,UB0))
            F0 = np.array(np.array(data_dict['F0']).reshape(M,UB0))
            G1 = np.array(np.array(data_dict['G1']).reshape(UB1,UB1))
            F1 = np.array(np.array(data_dict['F1']).reshape(M,UB1))
            G0, F0, N0=self.system_matrix(G0, F0, M, UB=UB0)
            G1, F1, N1=self.system_matrix(G1, F1, M, UB=UB1)
            print(f"Hidden_State Dimension of System 1 is:{N0}.")
            print(f"Hidden_State Dimension of System 2 is:{N1}.")
            print(f"System matrices are G0: \n {G0} and \n F0:\n{F0}.")
            print(f"System matrices are G1: \n {G1} and \n F1:\n{F1}.")
            if np.sum(np.array(data_dict['G0']) == 0) >= (N0*N0+N0*M)*thresh and np.sum(np.array(data_dict['F0']) == 0) >= (N0*N0+N0*M)*thresh and np.sum(np.array(data_dict['G1']) == 0) >= (N1*N1+N1*M)*thresh and np.sum(np.array(data_dict['F1']) == 0) >= (N1*N1+N1*M)*thresh:
              print("The System Matrices Is Sparse.")
              validation = 0
            else:
              print("The System Matrices Is Dense!")
              validation = 1
        else:
            idx = np.random.randint(0,2,len(data_in))
            K = [data_in[i] for i in np.where(idx==0)[0]]  #idx = 0
            L = [data_in[i] for i in np.where(idx==1)[0]]  #idx = 1
            print(len(K),len(L))
            # print(K[0].shape)

            ### EM ###
            for i in range(data_in.shape[1]):
                label_out= deepcopy(idx)
                print(i, "start: idx_=", label_out)

                if norm == True:
                  model = self.SimCom_norm(K,L,T=T,M=M, N=N, reg = 270,option='bonmin')
                else:
                  model = self.SimCom(K,L,T=T,M=M, N=N, reg = 270,option='bonmin')
                pred_k = model.f # index obj, model.f[i].value
                pred_l = model.ff

                for n in range(len(data_in)):
                    cost_k = sum((data_in[n][i,m]-pred_k[m,i].value)**2 for m in range(M) for i in range(T))
                    cost_l = sum((data_in[n][i,m]-pred_l[m,i].value)**2 for m in range(M) for i in range(T))
                    # print(cost_k, cost_l)
                    if cost_k < cost_l: # K cluster --> 0
                        idx[n] = 0
                    else:
                        idx[n] = 1

                print(i, "changed: idx=", idx)
                if np.array_equal(label_out,idx):
                    print(label_out, "end")
                    break
                else:
                    label_out = deepcopy(idx)
                    K = [data_in[i] for i in np.where(idx==0)[0]]  #idx = 0
                    L = [data_in[i] for i in np.where(idx==1)[0]]  #idx = 1
                    print('K:',len(K),'\n','L:',len(L))

        f1_1 = f1_score(label, label_out)
        f1_2 = f1_score(label, 1-label_out)

        if f1_1>f1_2:
            f1 = f1_1
        else:
            f1 = f1_2

        print('f1-score:',f1)

        if method == 'IF-Gurobi':
            return f1, validation
        else:
            return f1

## Utils

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import arff
import random
import re
import os
import functools
import logging

class utils_MIP4Cluster():

    def __init__(self):
        super().__init__()

    def summary_reg(self, folder_path):
        data_dict = {
            "validation": {},
            "countvalidation": {},
            "std": {},
            "mean": {},
            "duration": {}
        }

        pattern = re.compile(r'_(\d+(?:\.\d+)?)\.npy$')
        for filename in os.listdir(folder_path):
            if filename.endswith(".npy"):
                file_path = os.path.join(folder_path, filename)
                match = pattern.search(filename)
                if match:
                    key = match.group(1)
                    if "validation" in filename:
                        data_dict["validation"][key] = np.load(file_path)
                        data_dict["countvalidation"][key] = sum(1 for x in data_dict["validation"][key] if x != 0)
                    elif "std" in filename:
                        data_dict["std"][key] = np.load(file_path)
                    elif "mean" in filename:
                        data_dict["mean"][key] = np.load(file_path)
                    elif "duration" in filename:
                        data_dict["duration"][key] = np.load(file_path)

        for category in data_dict:
            data_dict[category] = dict(sorted(data_dict[category].items()))
        for category, values in data_dict.items():
            print(f"\nCategory: {category} ")
            for key, data in values.items():
                print(f"{key}: {data}")

    def log_exceptions(function):
        """
        A decorator that wraps the passed in function and logs
        exceptions should one occur
        """
        @functools.wraps(function)
        def wrapper(*args, **kwargs):
            try:
                return function(*args, **kwargs)
            except:
                # log the exception
                err = "There was an exception in  "
                err += function.__name__
                logging.exception(err)
                # re-raise the exception
                raise
        return wrapper

    def _create_recursive(parent_name, element):
        if isinstance(element, DictConfig):
            strings = []
            for k, v in element.items():
                if isinstance(v, DictConfig) or isinstance(v, ListConfig):
                    strings.append(_create_recursive(f'{parent_name}.{k}', v))
                else:
                    strings.append(f"{parent_name}.{k} = '{v}'")
            return ' and '.join(strings)
        elif isinstance(element, ListConfig):
            assert False, 'Lists not supported - cannot make OR query'
        else:
            return f"{parent_name} = '{element}'"
    '''
    def create_mlflow_query_string(params: DictConfig, finished=True):
        query_strings = []
        if finished:
            query_strings.append("status = 'FINISHED'")
        for param_name, element in params.items():
            query_strings.append(_create_recursive(f'params.{param_name}', element))
        return ' and '.join(query_strings)


    def log_system_info(hydra_config: DictConfig):
        mem_per_cpu = hydra_config.launcher.get("mem_per_cpu")
        if mem_per_cpu is not None:
            log_param('slurm.mem_per_cpu', mem_per_cpu)

        cpus_per_task = hydra_config.launcher.get("cpus_per_task")
        if cpus_per_task is not None:
            log_param('slurm.cpus_per_task', cpus_per_task)

        partition = hydra_config.launcher.get("partition")
        if partition is not None:
            log_param('slurm.partition', partition)
        log_param('system.hostname', os.uname()[1])
   '''

    def log_params_from_omegaconf_dict(params, only_keys=None):
        for param_name, element in params.items():
            if only_keys is None or param_name in only_keys:
                _explore_recursive(param_name, element)

    def _explore_recursive(parent_name, element):
        if isinstance(element, DictConfig):
            for k, v in element.items():
                if isinstance(v, DictConfig) or isinstance(v, ListConfig):
                    _explore_recursive(f'{parent_name}.{k}', v)
                else:
                    mlflow.log_param(f'{parent_name}.{k}', v)
        elif isinstance(element, ListConfig):
            for i, v in enumerate(element):
                mlflow.log_param(f'{parent_name}.{i}', v)
        else:
            mlflow.log_param(parent_name, element)

    def dynamic_generate(self, g,f_dash,proc_noise_std,obs_noise_std,inputs,T):
        import sys
        sys.path.append('../')
        from typing import List
        from copy import deepcopy
        import numpy as np
        import pandas as pd
        from math import sqrt
        from sklearn.preprocessing import scale, normalize, MinMaxScaler
        from scipy.spatial.distance import pdist, squareform
        from scipy.io import arff
        import logging

        #from inputlds import *

        import matplotlib.pyplot as plt
        """
        parametrs for dynamical_syste(A,B,C,D, **kwargs)
        input: A,B,C,D, **kwargs
        phi_(t+1) = A*phi_t + B*X_t + w_(t+1)
            Y_t = C*phi_t + D*X_t + v_t
        A--> g:[n,n]
        B--> B:[n,d] matrix for inputs, d is the dimension of inputs
        C--> f_dash:[m,n]
        D--> D:[m,d] matrix for inputs, d is the dimension of inputs
        n is the dimension of hidden states (phi: [T,n]);
        m is the dimension of observations (Y: [T,m]);
        d is the dimension of inputs (X: [T,d]).

        """
        n=len(g)
        m=len(f_dash)
        if inputs == 0: # no inputs
            inputs = np.zeros((m,T))
        dim = len(inputs) # dimension of inputs
        ds1 = dynamical_system(g,np.zeros((n,dim)),f_dash,np.zeros((m,dim)),
                process_noise='gaussian',
                observation_noise='gaussian',
                process_noise_std=proc_noise_std,
                observation_noise_std=obs_noise_std)

        h0=np.ones(ds1.d) # initial state
        ds1.solve(h0=h0, inputs=inputs, T=T)
        return np.asarray(ds1.outputs).reshape(T,m) #.tolist()

    def data_generation(self, g,f_dash,pro_rang,obs_rang,T,S):
        import logging
        logging.info("开始生成数据...")
        proL=len(pro_rang)
        obsL=len(obs_rang)
        file_name =[]
        for gg in range(len(g)):
            n=gg+2
            m=2
            cluster_1 = []
            cluster_2 = []
            for i in range(proL):
                for j in range(obsL):
                    proc_noise_std=pro_rang[i]
                    obs_noise_std=obs_rang[j]
                    # Generate data
                    # inputs = np.zeros((2,T))
                    inputs = 0
                    for k in range(S):
                        data_1 = self.dynamic_generate(g[gg, 0],f_dash[gg][0],proc_noise_std,obs_noise_std,inputs,T)
                        cluster_1.append(data_1)

                        data_2 = self.dynamic_generate(g[gg, 1],f_dash[gg][1],proc_noise_std,obs_noise_std,inputs,T)
                        cluster_2.append(data_2)

            Y = np.concatenate((np.array(cluster_1),np.array(cluster_2)),axis=0)
            Y_label = np.concatenate((np.zeros(len(cluster_1)),np.ones(len(cluster_2))),axis=0)
            print(Y.shape, Y_label.shape)
            data = Y.reshape(320,-1)
            #label = Y_label.reshape(32,)
            f_name = f'./data/{n}_{m}_test.npy'
            with open(f_name, 'wb') as f:
                np.save(f, data)
            file_name.append(f_name)
        return file_name

    def datacleaning(self, data_dir, name, S, I, T, M, J):
        import logging
        logging.info("开始读取数据...")
        logging.info("数据预处理开始...")
        if name =='lds':
            path_list = ['./data/2_2_test.npy', './data/3_2_test.npy','./data/4_2_test.npy']
            X__=[]
            L__=[]
            for path in path_list:
                data_X = np.load(path)
                data_X = data_X.reshape(2*S, I, T, M)

                X_ = np.zeros((S, 2*I, T, M))
                L_ = np.zeros((S, 2*I))
                for s in range(S):
                    xx_1 = data_X[s]
                    xx_2 = data_X[s+S]
                    X_[s] = np.concatenate((xx_1,xx_2),axis=0).reshape(2*I, T, M)
                    L_[s] = np.concatenate((np.zeros(I),np.ones(I)),axis=0)
                X__.append(X_)
                L__.append(L_)
            X = np.array(X__).reshape(3, S, 2*I, T, M) #3, 10, 32, 20, 2
            label = np.array(L__).reshape(3, S, 2*I)
            print("Final data shape:", X.shape)
            print("Labels shape:", label.shape)
            np.save("./data/lds_X.csv", X)
            return X,label

        elif name =='ecg':
            def a2p(path):
                data, meta = arff.loadarff(path)
                df = pd.DataFrame(data)
                return df

            trainpath = "./data/ECG5000_TRAIN.arff"
            X_data = a2p(trainpath)
            print(X_data.target.value_counts())
            X_data.head()
            X_1 = X_data[X_data.target==b'1'].iloc[:,:-1].values
            X_2 = X_data[X_data.target==b'2'].iloc[:,:-1].values
            idx_1 = np.arange(len(X_1))
            idx_2 = np.arange(len(X_2))
            iidx_1 = random.sample(sorted(idx_1), I)
            iidx_2 = random.sample(sorted(idx_2), I)
            np.random.shuffle(iidx_1)
            np.random.shuffle(iidx_2)
            XX_1 = X_1[iidx_1]
            XX_2 = X_2[iidx_2]
            X = np.concatenate((XX_1, XX_2), axis=0).reshape(S, 2*I,T,M)#1*30*140*1
            label=[]
            for s in range(S):
               L_= np.concatenate((np.zeros(I),np.ones(I)),axis=0)
               label.append(L_)
            label = np.array(label).reshape(S, 2*I)
            print(X)
            print("Final data shape:", X.shape)
            print("Labels shape:", label.shape)
            # np.save("./data/ecg_X.csv", X)
            return X, label

        elif name =='biosensors':
            df = pd.read_csv('./data/biosensors.csv')
            target_columns = ['patient','risk-group', 'bmi-(body-mass-index)', 'pbf-(percent-body-fat)', 'vfl-(visceral-fat-level)','site','gender','visit','date-of-birth','exam-date']
            df = df[target_columns].dropna(subset=target_columns).reset_index(drop=True)
            df.columns = ['patient','risk-group','BMI', 'PBF', 'VFL','site','gender','visit','date-of-birth','exam-date']
            risk_mapping = {'Normal': 0, 'I': 1, 'II': 2}
            df['risk-group-numeric'] = df['risk-group'].map(risk_mapping)
            df['VFL'] = df['VFL'].str.extract('(\d+)').astype(float)
            #print(df)

            # Train data
            patients = df['patient'].unique()
            num_patients = len(patients)
            time_steps = 3  # 每个患者有3次随访
            num_features = 3  # 选择3个特征：BMI, PBF, VFL

            X_data = []
            for i, patient in enumerate(patients):
                patient_data = df[df['patient'] == patient]#.sort_values('visit')
                #print(patient_data)
                if patient_data.shape[0] ==3:
                    X_data.append(patient_data[['BMI','PBF','VFL']].values)
            #print(X_data)

            # Label
            risk_sum = (df.groupby('patient')
                      .filter(lambda x: len(x) == 3)  # 只保留有≥3次随访的患者
                      .groupby('patient')['risk-group-numeric']
                      .sum()
                      .reset_index())
            #print(len(risk_sum))
            risk_sum.columns = ['patient', 'total_risk_score']
            risk_sum['label'] = (risk_sum['total_risk_score'] >= 2).astype(int)
            label_data = risk_sum['label'].values
            #print(label_data)
            X=np.array(X_data).reshape(6, 23, 3, 3)
            label=np.array(label_data).reshape(6, 23)
            print("Final data shape:", X.shape)
            print("Labels shape:", label.shape)
            np.save("./data/biosensors_X.csv", X)
            return X, label

        elif name =='seizure-eeg':
            data = np.load("./data/eeg_seizure_Train_normalized.npy")
            positive_indices = {
                'chb01': [3, 4, 15, 16, 18, 21, 26],
                'chb03': [42, 43, 44, 45, 75, 76, 77],
                'chb05': [85, 92, 95, 96, 101, 96, 92]
            }
            negative_indices = {
                'chb01': [6, 10, 22, 25, 30, 35, 40],
                'chb03': [49, 51, 55, 58, 60, 66, 71],
                'chb05': [81, 89, 104, 109, 113, 116, 98]
            }
            def get_samples(patient, positive_indices, negative_indices):
                pos = positive_indices.get(patient, [])
                neg = negative_indices.get(patient, [])

                pos_data = data[pos]
                neg_data = data[neg]

                all_patient_data = np.concatenate([pos_data, neg_data])

                labels = np.concatenate([np.ones(len(pos_data)), np.zeros(len(neg_data))])
                return all_patient_data, labels

            chb01_data, chb01_labels = get_samples('chb01', positive_indices, negative_indices)
            chb03_data, chb03_labels = get_samples('chb03', positive_indices, negative_indices)
            chb05_data, chb05_labels = get_samples('chb05', positive_indices, negative_indices)
            X = np.array([chb01_data, chb03_data, chb05_data], object) # Use object dtype to store arrays of different shapes
            label = np.array([chb01_labels, chb03_labels, chb05_labels], object) # Use object dtype to store arrays of different shapes

            print("Final data shape:", X.shape)
            print("Labels shape:", label.shape)
            np.save("./data/eeg_seizure_X.csv", X)
            return X, label

        elif data_dir!=[]:
            data_dict = {}
            for file_name in os.listdir(data_dir):
                if file_name.endswith(".npy"):  # 只处理 .npy 文件
                    file_path = os.path.join(data_dir, file_name)
                    key_name = file_name.replace('.npy', '')  # 去掉后缀作为键
                    data_dict[key_name] = np.load(file_path, allow_pickle=True)
                    if len(data_dict[key_name].shape) ==3:
                        S, T, M = data_dict[key_name].shape
                    elif len(data_dict[key_name].shape) ==4:
                        S, I_, T, M = data_dict[key_name].shape
                    else:
                        print(f"Error processing {file_name}")

            if data_dict.shape ==(2, 3):
                Y = np.concatenate((np.array(data_dict[0]),np.array(data_dict[1])),axis=0)
                Y_label = np.concatenate((np.zeros(len(data_dict[0])),np.ones(len(data_dict[1]))),axis=0)
                print(Y.shape, Y_label.shape)
                data = Y.reshape(S*I*2,-1)
                with open('{name}_test.npy', 'wb') as f:
                    np.save(f, data)

            elif data_dict.shape ==(4, 1):
                label=np.concatenate((np.zeros(I_/2),np.ones(I_/2)),axis=0)

            else:
                print("Cannot deal with the given data type, please contact owner!")

        else:
            print("Wrong data type!")

    def plot_MIF4cluster_methods(self, path, methods, name, cutdown ):
        import numpy as np
        import matplotlib.pyplot as plt
        from mpl_toolkits.axisartist.parasite_axes import HostAxes, ParasiteAxes
        from matplotlib.backends.backend_pdf import PdfPages

        colormap=('#4292c6','#696969', '#CD5C5C' ,'#FFD700', '#6B8E23')
        labels=methods
        # load results on Synthetic dataset
        mean = []
        std = []
        for method in methods:
            if cutdown==True:
                mean.append(np.load(f"{path}f1_{method}_{name}_mean_cd.npy"))
                std.append(np.load(f"{path}f1_{method}_{name}_std_cd.npy"))
            else:
                mean.append(np.load(f"{path}f1_{method}_{name}_mean.npy"))
                std.append(np.load(f"{path}f1_{method}_{name}_std.npy"))
        mean = np.concatenate(mean).reshape(5,3) # Concatenate along axis 0 by default
        std = np.concatenate(std).reshape(5,3) # Concatenate along axis 0 by default
        print([mean.shape, std.shape])
        ##############################################
        ############# plot for n = 2,3,4 #############
        ########## fft, dtw, ind, emh, gurobi ########
        ##############################################

        met_range=[*range(5)]
        nx_range=np.arange(2,5)

        # fig = plt.figure(figsize=(10,6), dpi=100)
        # mean = np.array(results[f"{method}_{name}_mean"] for method in methods)
        # std = np.array(results[f"{method}_{name}_std"] for method in methods)
        fig, ax = plt.subplots(figsize=(8,5), dpi=200)
        width=0.1
        for m in met_range:
            x=nx_range+m*width
            y=mean[m,:]
            y_error=1.96*std[m,:]/np.sqrt(50)
            ax.plot(x,y, color=colormap[m],label=labels[m])
            ax.errorbar(x, y, yerr = y_error, fmt ='.', color=colormap[m],capsize=4,capthick=2) #,align='edge'

        # plt.legend(bbox_to_anchor=(1.02,1.25),fontsize=14,frameon=False,ncol=2)
        plt.legend(fontsize=14,frameon=False,ncol=1,loc='lower right')
        plt.yticks(ticks=[0.0,0.25,0.5,0.75,1.0],labels=[0.0,0.25,0.5,0.75,1.0],fontsize=14)
        plt.xticks(ticks=[2.3,3.3,4.3],labels=[2,3,4],fontsize=14)
        plt.xlabel('dimensions of system matrices '+r'$n$',fontsize=16)
        plt.ylabel('F1 score',fontsize=16)
        # plt.savefig('prettyplot_higherdim.pdf', bbox_inches='tight')





## MIP4Cluster Estimator

In [ ]:
class MIP4Cluster:
    """Estimator based on NCPOP Regressor"""

    def __init__(self):
        super().__init__()

    def setup_environment(self,method):
        import sys
        import importlib
        """
        Set up the environment by installing necessary packages and configuring paths.

        Parameters:
        method : str - The method being used (e.g., 'IF-Gurobi', 'IF', etc.)
        """

        # Install required packages for 'IF-Gurobi'
        if method == 'IF-Gurobi':
            if not importlib.util.find_spec("gurobipy") or  not importlib.util.find_spec("sklearn") or not importlib.util.find_spec("seaborn"):
                !pip install scikit-learn seaborn
                from sklearn.metrics import f1_score
                from scipy.io import arff
                !pip install gurobipy
            import gurobipy as gp
            from sklearn.metrics import f1_score
            from scipy.io import arff

            # Set up Gurobi license path (update this as needed)
            os.environ['GRB_LICENSE_FILE'] = '/content/drive/MyDrive/gurobi.lic'

        elif method in['EM', 'IF']:
            if 'google.colab' in sys.modules:
                !pip install idaes-pse --pre
                !idaes get-extensions --to ./bin
                os.environ['PATH'] += ':bin'
        else:
            if not importlib.util.find_spec("tslearn"):
                !pip install tslearn

    def run_experiment(self, data_in, label_in, method, name, S, I, T, M, N, J, T_start_rand, T_rate, reg):
        # S, I, T, M, N, N_Data
        f1_list = []
        validation = []
        duration = []
        for nn in N:
            n = int(nn[0])
            t_start = time.time()
            for mm in M:
              m = int(mm[0])
              for tt in T:
                t = int(tt[0])
                for s in range(S):
                    for j in range(J):
                          np.random.seed(s+j+30)
                          if I >= 10:
                              I_size = np.random.randint(I, I*3/2)
                          else:
                              I_size = np.random.randint(I*3/2, I*2)
                          sampl = np.random.choice(2*I, size=I_size, replace=False)
                          label = label_in[s][np.ix_(sampl)]
                          label = np.array(list(label))
                          T_start = np.random.randint(T_start_rand, T_start_rand+T_rate)
                          # print([s], sampl, t, m)
                          X = data_in[s][np.ix_(sampl, range(T_start, T_start + t), range(m))]
                          print(X.shape, label.shape)

                          f1_result, valid_result = MIP_IF().MIP_estimate(X, label, method, n, name, reg, s+j+30, thresh=0.25)
                          print(f"n:{n}. regularization:{reg}. F1:{f1_result}")
                          validation.append(valid_result)
                          f1_list.append(f1_result)

            duration.append(time.time() - t_start)

        return f1_list, duration, validation

    def Test_MIP4Cluster(self, data_in, label_in, met, S, I, T, M, name, N, J, T_start_rand, T_rate, regularization, MTS=True,option='bonmin',norm=True, thresh=0.25):
        import os
        import sys
        import time
        import importlib.util

        if not os.path.exists(f'./Result_{name}'):
            os.makedirs(f'./Result_{name}')
        if not os.path.exists(f'./Result_{name}'):
            os.makedirs(f'./Result_{name}')
        print(data_in.shape, label_in.shape)
        data_num = data_in.shape[0]
        for method in met:
            if method in ['IF-Gurobi', 'IF', 'EM', 'FFT', 'DTW']:
                self.setup_environment(method)

                if all(len(dim.shape) <= 1 for dim in [label_in, T, M, N]):  # Corrected the condition
                    raise ValueError("Cannot generate two dims result.")
                else:
                    # Reshape
                    print(T.shape, N.shape)
                    if T.shape[0] <= 1 and N.shape[0] <= 1:
                        raise ValueError("Cannot reshape.")
                    elif T.shape[0] <= 1:
                        list_shape = (N.shape[0], S*J) # 3*20
                        shape = (1, N.shape[0])
                        axis =1
                    elif len(N) <= 1:
                        list_shape = (T.shape[0], S*J) # 5*20
                        shape = (1,N.shape[0])
                        axis =1
                    else:
                        list_shape = (T.shape[0], N.shape[0], S*J) # 5*3*20
                        shape = (T.shape[0], N.shape[0])
                        axis =2

                ###################################################### Regularization ##############################################
                if regularization.shape[0]>1:
                    if method == 'IF-Gurobi':
                        for r in regularization:
                            reg=int(r)
                            f1_list, duration, validation  = self.run_experiment(data_in, label_in, method, name, S, I, T, M, N, J, T_start_rand, T_rate, reg)
                            f1 = np.array(f1_list).reshape(list_shape)
                            f1_mean = np.mean(f1, axis=axis)
                            f1_std = np.std(f1, axis=axis)
                            duration = np.array(duration).reshape(shape)
                            validation = np.array(validation).reshape(list_shape)
                            count_validation = np.sum(validation == 1)
                            print(f'f1_{method}_{name}_mean_reg{reg} is: \n {f1_mean}, '
                                  f'f1_{method}_{name}_std_reg{reg} is: \n {f1_std}, '
                                  f'duration_{method}_{name}_reg{reg} is: \n {duration}, '
                                  f'validation_{method}_{name}_reg{reg} is: \n {validation},'
                                  f'Nonzero System Matrices are: {count_validation}.')
                            save_paths = [
                                (f'./Result_{name}/f1_{method}_{name}_reg{reg}.npy', f1),
                                (f'./Result_{name}/f1_{method}_{name}_mean_reg{reg}.npy', f1_mean),
                                (f'./Result_{name}/f1_{method}_{name}_std_reg{reg}.npy', f1_std),
                                (f'./Result_{name}/duration_{method}_{name}_reg{reg}.npy', duration),
                                (f'./Result_{name}/validation_{method}_{name}_reg{reg}.npy', validation)
                            ]
                            for path, data in save_paths:
                                with open(path, 'wb') as f:
                                    np.save(f, data)
                        break

                    else:
                        print('Warning: Cannot compare regularization for other method rather than IF-Gurobi.')

                ############################################ 'IF-Gurobi', 'IF', 'EM', 'FFT', 'DTW' #############################################
                else:
                    reg = regularization[0]
                    f1_list = []
                    validation = []
                    duration = []
                    # S, I, T, M, N
                    for nn in N:
                        n = int(nn[0])
                        t_start = time.time()
                        for mm in M:
                          m = int(mm[0])
                          for tt in T:
                            t = int(tt[0])
                            for s in range(S):
                                for j in range(J):
                                    np.random.seed(s+j+30)
                                    if I >= 10:
                                        I_size = np.random.randint(I, I*3/2)
                                    else:
                                        I_size = np.random.randint(I*3/2, I*2)
                                    sampl = np.random.choice(2*I, size=I_size, replace=False)
                                    label = label_in[s][np.ix_(sampl)]
                                    label = np.array(list(label))
                                    T_start = np.random.randint(T_start_rand, T_start_rand+T_rate)
                                    # print([s], sampl, t, m)
                                    X = data_in[s][np.ix_(sampl, range(T_start, T_start + t), range(m))]
                                    print(X.shape, label.shape)
                                    if j == 0:
                                        is_plot = True
                                    else:
                                        is_plot = False
                                    # print(f"n:{n}")
                                    if method == 'IF-Gurobi':
                                          # Test IF-Gurobi
                                          f1_result, valid_result = MIP_IF().MIP_estimate(X, label, method, n, name, reg, seed=s+j+30, thresh=0.25)
                                          print(f"n:{n}, regularization:{reg}, {method}_F1:{f1_result}.")
                                          validation.append(valid_result)

                                    elif method == 'IF':
                                          # Test IF-Bonmin
                                          f1_result = MIP_IF().MIP_estimate(X, label, method, n, name, reg, seed=s+j+30)
                                          print(f"n:{n},{method}_F1:{f1_result}.")

                                    elif method == 'EM':
                                          # Test EM Heuristic
                                          f1_result = MIP_IF().MIP_estimate(X, label, method, n, name, reg, seed=s+j+30, MTS=True, option='bonmin', norm=True)
                                          print(f"n:{n}.{method}_F1:{f1_result}")

                                    elif method =='DTW':
                                          # Test DTW
                                          f1_result = MIP_IF().DTW_estimate(X, label, seed=s+j+30, is_plot =is_plot)
                                          print(f"{method}_F1:{f1_result}")

                                    else:
                                          # Test FFT
                                          f1_result = MIP_IF().FFT_estimate(X, label, seed=s+j+30, is_plot =is_plot)
                                          print(f"{method}_F1:{f1_result}")
                                    f1_list.append(f1_result)
                        duration.append(time.time() - t_start)
                f1 = np.array(f1_list).reshape(list_shape)
                f1_mean = np.mean(f1, axis=axis)
                f1_std = np.std(f1, axis=axis)
                duration = np.array(duration)#.reshape(shape)
                suffix = "_cd" if max(max(T)) < 10 and T.shape[0]!=1 and regularization.shape[0]==1 else ""

                print(f'f1_{method}_{name}_mean{suffix} is: \n {f1_mean}, '
                      f'f1_{method}_{name}_std{suffix} is: \n {f1_std}, '
                      f'duration_{method}_{name}{suffix} is: \n {duration}. ')
                save_paths = [
                    (f'./Result_{name}/f1_{method}_{name}{suffix}.npy', f1),
                    (f'./Result_{name}/f1_{method}_{name}_mean{suffix}.npy', f1_mean),
                    (f'./Result_{name}/f1_{method}_{name}_std{suffix}.npy', f1_std),
                    (f'./Result_{name}/duration_{method}_{name}{suffix}.npy', duration)
                ]
                for path, data in save_paths:
                    with open(path, 'wb') as f:
                        np.save(f, data)

        return f1_mean, f1_std, duration
        print(f'Check result under route ./Result_{name}/ .')



# Tests

1. log
2. 结果不是0.9
3. EEG结果

## Test Synthetic Data

### **Data Generator**

In [ ]:
if __name__ == '__main__':
    # Self-setting
    S_len, T_len= 10, 20
    # Collect the nrmse value for each experiment
    pro_rang = np.arange(0.02,0.1,0.02)
    obs_rang = np.arange(0.02,0.1,0.02)

    g = np.array([[0.8*np.matrix([[0.9,0.2],[0.1,0.1]]), 0.8*np.matrix([[0.8,0.2],[0.2,0.1]])],
                  [0.6*np.matrix([[1.0,0.8,0.8],[0.6,0.1,0.2],[0.3,0.2,0.2]]), 0.6*np.matrix([[1.0,1.0,0.6],[0.7,0.2,0.2],[0.2,0.1,0.1]])],
                  [np.matrix([[0.9,0.8,0.5,0.2],[0.9,0.1,0.3,0.4],[0.8,0.2,0.1,0.1],[0.1,0.1,0.1,0.7]])*0.4, np.matrix([[1.0,0.8,0.5,0.3],[0.6,0.2,0.3,0.4],[0.8,0.2,0.3,0.1],[0.2,0.2,0.3,0.7]])*0.4]], dtype=object)

    f_dash = {0: [0.8 * np.array([[1.0, 1.0], [0.2, 0.2]]), 0.8 * np.array([[0.8, 1.0], [0.1, 0.2]])],
              1: [0.6 * np.array([[0.7, 0.4, 0.3], [0.2, 0.6, 0.2]]), 0.6 * np.array([[0.5, 0.4, 0.1], [0.2, 0.5, 0.1]])],
              2: [np.array([[0.2, 0.5, 0.1, 0.1], [0.8, 0.6, 0.1, 0.1]]) * 0.4, np.array([[0.2, 0.4, 0.1, 0.1], [0.6, 0.2, 0.2, 0.2]]) * 0.4]}

    data_dir=utils_MIP4Cluster().data_generation(g,f_dash,pro_rang,obs_rang,T_len,S_len)
    print(data_dir)

(320, 20, 2) (320,)
(320, 20, 2) (320,)
(320, 20, 2) (320,)
['./data/2_2_test.npy', './data/3_2_test.npy', './data/4_2_test.npy']


### Reg

In [ ]:
# Don't know where to start, try here👇🏻:
if __name__ == '__main__':
    # Self-setting
    S_len, I_len, T_len, F_len =10, 16, 20, 2
    J = 5
    name = "lds"
    hiddenstates =[2,3,4]
    windows = [3]
    feature_selection = [2]
    method= ['IF-Gurobi']
    reg = np.array(range(20, 100, 50))

    # Generate Data
    # data_dir=utils_MIP4Cluster().data_generation(g,f_dash,pro_rang,obs_rang,T_len,S_len)
    data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
    window = np.array([[w] for w in windows])# Timeset
    F = np.array([[f] for f in feature_selection])# Feature
    N = np.array([[n] for n in hiddenstates])
    T_start = 5
    T_rate = 5
    i =0
    num =2+i
    data_X = data[i]
    label_X = label[i]
    MIP4Cluster().Test_MIP4Cluster(data_X, label_X, method, S_len, I_len, window, F, f"lds{num}", N, J, T_start, T_rate, reg, thresh=0.25)

Final data shape: (3, 10, 32, 20, 2)
Labels shape: (3, 10, 32)
(10, 32, 20, 2) (10, 32)
(1, 1) (2, 1)
(21, 3, 2) (21,)
m,t: 2 3
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2611849
Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Set parameter TimeLimit to value 360
5 5 2 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  360
NonConvex  2

Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Optimize a model with 4 rows, 357 columns and 84 nonzeros
Model fingerprint: 0x402589e3
Model has 196 quadratic objective terms
Model has 196 quadratic constraints
Variable types: 336 continuous, 21 integer (21 binary)
Coefficient statistics:


GurobiError: Unable to retrieve attribute 'x'

### **Test-log**

In [ ]:
import logging
from logging.handlers import RotatingFileHandler
import time
import os
from pathlib import Path
from omegaconf import DictConfig
#import hydra
#from hydra.core.hydra_config import HydraConfig
#import tracking_utils as tu
import numpy as np

# 初始化日志系统
def setup_logging(output_dir):
    """配置完整的日志系统"""
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # 文件日志（自动轮转）
    log_file = Path(output_dir) / "experiment.log"
    file_handler = RotatingFileHandler(
        log_file,
        maxBytes=10*1024*1024,  # 10MB
        backupCount=5,
        encoding='utf-8'
    )
    file_handler.setFormatter(logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    ))

    # 控制台日志
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter('%(message)s'))

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

def start_experiment(cfg: DictConfig) -> None:
    # 获取输出目录并创建日志
    output_dir = HydraConfig.get().runtime.output_dir
    logger = setup_logging(output_dir)
    experiment_logger = logging.getLogger(__name__)

    # 初始化MLflow跟踪
    tu.set_tracking(cfg.tracking, cfg.experiment)

    # 记录配置
    conf_yaml = OmegaConf.to_yaml(cfg)
    experiment_logger.info("实验配置:\n%s", conf_yaml)

    with tu.start_run() as run:
        # 记录参数和系统信息
        log_params_from_omegaconf_dict(cfg, {'solver', 'problem'})
        log_system_info(HydraConfig.get())

        # 保存配置到文件
        config_file = Path(output_dir) / "config.yaml"
        with open(config_file, 'w', encoding='utf-8') as f:
            f.write(conf_yaml)
        run.log_artifact(str(config_file))

        try:
            # 实验参数
            params = {
                'S_len': 10, 'I_len': 16, 'T_len': 20, 'F_len': 2,
                'J': 5, 'name': "lds", 'hiddenstates': [5, 2],
                'windows': [3], 'feature_selection': [2],
                'reg': np.array([270]), 'methods': ['IF-Gurobi','IF','EM','FFT','DTW'],
                'T_start': 5, 'T_rate': 5, 'thresh': 0.25
            }

            # 数据生成
            experiment_logger.info("开始生成数据...")
            start_time = time.time()
            data, label = utils_MIP4Cluster().datacleaning(
                [], params['name'], params['S_len'], params['I_len'],
                params['T_len'], params['F_len'], params['J']
            )
            experiment_logger.info(
                "数据生成完成 - 耗时: %.2f秒 | 数据形状: %s",
                time.time() - start_time,
                data.shape
            )

            # 运行实验
            for i in range(data.shape[0]):
                num = 2 + i
                data_X = data[i]
                label_X = label[i]

                experiment_logger.info("="*50)
                experiment_logger.info("开始处理数据集 %d/%d", num, data.shape[0])

                # 记录数据集信息
                experiment_logger.debug(
                    "数据集统计 - 均值: %.4f ± %.4f",
                    np.mean(data_X),
                    np.std(data_X)
                )

                # 运行求解器
                solver_start = time.time()
                experiment_logger.info("启动求解器: %s", params['methods'])

                try:
                    f1_mean, f1_std, duration = MIP4Cluster().Test_MIP4Cluster(
                        data_X, label_X, params['methods'],
                        params['S_len'], params['I_len'],
                        np.array([[w] for w in params['windows']]),
                        np.array([[f] for f in params['feature_selection']]),
                        f"lds{num}",
                        np.array([[n] for n in params['hiddenstates']]),
                        params['J'], params['T_start'],
                        params['T_rate'], params['reg'],
                        thresh=params['thresh']
                    )

                    experiment_logger.info(
                        "求解完成 - 耗时: %.2f秒 | F1分数: %.3f ± %.3f",
                        duration, f1_mean, f1_std
                    )

                    # 记录到MLflow
                    run.log_metric(f"dataset_{num}_f1_mean", f1_mean)
                    run.log_metric(f"dataset_{num}_f1_std", f1_std)
                    run.log_metric(f"dataset_{num}_duration", duration)

                except Exception as e:
                    experiment_logger.error(
                        "处理数据集 %d 时出错: %s",
                        num, str(e),
                        exc_info=True
                    )
                    run.log_text(f"Dataset {num} error: {str(e)}", f"error_{num}.txt")
                    continue

        except Exception as e:
            experiment_logger.critical(
                "实验遇到致命错误: %s",
                str(e),
                exc_info=True
            )
            run.log_text(f"Critical error: {str(e)}", "critical_error.txt")
            raise
        finally:
            experiment_logger.info("实验结束")
            logging.shutdown()

if __name__ == "__main__":
    start_experiment()

ModuleNotFoundError: No module named 'omegaconf'

In [ ]:
# Don't know where to start, try here👇🏻:
import tracking_utils as tu
logger = logging.getLogger(__name__)

@hydra.main(version_base=None,  config_path="./experiments_conf", config_name="config")
@log_exceptions
def start_experiment(cfg: DictConfig) -> None:
    tu.set_tracking(cfg.tracking, cfg.experiment) #mlflow.set_experiment(experiment_name=cfg.experiment)
    conf_yaml = OmegaConf.to_yaml(cfg)
    print(conf_yaml)
    app_config = OmegaConf.to_container(cfg)
    seed = cfg.problem.seed


    utils.set_random_seed(seed)
    with tu.start_run() as run:
        #log_params_from_omegaconf_dict(cfg)
        log_params_from_omegaconf_dict(cfg, {'solver', 'problem'})
        log_system_info(HydraConfig.get())
        note = cfg.get('note','')
        run.log_param('note', note)
        output_dir = hydra.core.hydra_config.HydraConfig.get().runtime.output_dir
        run.log_text(output_dir, 'work_dir.txt')
        logger.info(f'Starting experiment in {output_dir}')
        with open(join(output_dir, 'config.yaml'), 'w') as f:
            f.write(conf_yaml)
        run.log_artifact(join(output_dir, 'config.yaml'))

        try:
            S_len, I_len, T_len, F_len =10, 16, 20, 2
            J = 5
            name = "lds"
            hiddenstates =[5,2]
            windows = [3]
            feature_selection = [2]
            # regularization set
            # method= ['IF-Gurobi']
            # reg = np.array(range(20, 100, 50))
            reg = np.array([270])

            # Generate Data
            # data_dir=utils_MIP4Cluster().data_generation(g,f_dash,pro_rang,obs_rang,T_len,S_len)
            data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
            logger.info(f'DATA GENERATED')
            method= ['IF-Gurobi','IF','EM','FFT','DTW']
            window = np.array([[w] for w in windows])# Timeset
            F = np.array([[f] for f in feature_selection])# Feature
            N = np.array([[n] for n in hiddenstates])
            T_start = 5
            T_rate = 5
            for i in range(data.shape[0]):
                num =2+i
                data_X = data[i]
                label_X = label[i]
                logger.info(f'STARTING SOLVER')
                f1_mean, f1_std, duration = MIP4Cluster().Test_MIP4Cluster(data_X, label_X, method, S_len, I_len, window, F, f"lds{num}", N, J, T_start, T_rate, reg, thresh=0.25)

if __name__ == "__main__":
    start_experiment()

SyntaxError: expected 'except' or 'finally' block (<ipython-input-16-76fa213b96f8>, line 58)

In [ ]:
import logging
from logging.handlers import RotatingFileHandler  # 新增：更好的日志处理器
import pickle
import signal
import sys
import numpy as np

def setup_logging(name):
    """配置更健壮的日志记录"""
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    # 使用RotatingFileHandler防止日志过大
    file_handler = RotatingFileHandler(
        f'{name}_run.log',
        maxBytes=10*1024*1024,  # 10MB
        backupCount=5,
        encoding='utf-8'  # 确保中文支持
    )
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

    # 控制台输出
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # 确保立即刷新
    file_handler.flush()
    return logger

def save_params(params, filename):
    """保存参数到文件"""
    with open(filename, 'wb') as f:
        pickle.dump(params, f)
    logging.info(f"参数已保存到 {filename}")

def handle_interrupt(signum, frame):
    """改进的中断处理函数"""
    logger = logging.getLogger(params.get('name', 'default'))
    logger.warning(f"接收到中断信号 {signum}", extra={'interrupt': True})

    # 确保所有日志处理器都刷新
    for handler in logger.handlers:
        handler.flush()

    # 更优雅的退出方式
    sys.exit(1)

if __name__ == '__main__':
    # 初始化参数
    params = {
        'S_len': 10, 'I_len': 16, 'T_len': 20, 'F_len': 2,
        'J': 5, 'name': "lds", 'hiddenstates': [5, 2],
        'windows': [3], 'feature_selection': [2],
        'reg': np.array([270]), 'methods': ['IF-Gurobi','IF','EM','FFT','DTW'],
        'T_start': 5, 'T_rate': 5, 'thresh': 0.25
    }

    # 设置日志和中断处理
    logger = setup_logging(params['name'])

    # 注册信号处理器（放在logger初始化之后）
    signal.signal(signal.SIGINT, handle_interrupt)
    signal.signal(signal.SIGTERM, handle_interrupt)

    try:
        # 保存参数
        param_file = f"{params['name']}_params.pkl"
        save_params(params, param_file)

        # 生成数据
        logger.info("开始生成数据...")
        data, label = utils_MIP4Cluster().datacleaning([], params['name'],
                                                     params['S_len'], params['I_len'],
                                                     params['T_len'], params['F_len'],
                                                     params['J'])

        # 主处理循环
        for i in range(data.shape[0]):
            num = 2 + i
            data_X = data[i]
            label_X = label[i]
            logger.info(f"开始处理数据集 {num}...")

            try:
                MIP4Cluster().Test_MIP4Cluster(
                    data_X, label_X, params['methods'],
                    params['S_len'], params['I_len'],
                    window, F, f"{params['name']}{num}",
                    N, params['J'], params['T_start'],
                    params['T_rate'], params['reg'],
                    thresh=params['thresh']
                )
                logger.info(f"数据集 {num} 处理完成")

                # 显式刷新日志
                for handler in logger.handlers:
                    if hasattr(handler, 'flush'):
                        handler.flush()

            except Exception as e:
                logger.error(f"处理数据集 {num} 时出错: {str(e)}", exc_info=True)
                continue

    except Exception as e:
        logger.critical(f"主程序出错: {str(e)}", exc_info=True)
    finally:
        logger.info("程序结束")
        # 确保所有日志都写入
        logging.shutdown()

2025-04-02 13:12:33,431 - INFO - 开始生成数据...
2025-04-02 13:12:33,431 - INFO - 开始生成数据...
INFO:lds:开始生成数据...
2025-04-02 13:12:33,483 - INFO - 开始处理数据集 2...
2025-04-02 13:12:33,483 - INFO - 开始处理数据集 2...
INFO:lds:开始处理数据集 2...


Final data shape: (3, 10, 32, 20, 2)
Labels shape: (3, 10, 32)
(10, 32, 20, 2) (10, 32)
(1, 1) (2, 1)
(21, 3, 2) (21,)
m,t: 2 3
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2611849
Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Set parameter TimeLimit to value 360
5 5 2 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  360
NonConvex  2

Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Optimize a model with 4 rows, 357 columns and 84 nonzeros
Model fingerprint: 0x4e307d47
Model has 196 quadratic objective terms
Model has 196 quadratic constraints
Variable types: 336 continuous, 21 integer (21 binary)
Coefficient statistics:


2025-04-02 13:13:33,226 - WARNING - 接收到中断信号 2
2025-04-02 13:13:33,226 - WARNING - 接收到中断信号 2
2025-04-02 13:13:33,234 - INFO - 程序结束
2025-04-02 13:13:33,234 - INFO - 程序结束
INFO:lds:程序结束


THIS IS NOT OPTIMAL SOLUTION
Hidden_State Dimension of System 1 is:5.
Hidden_State Dimension of System 2 is:5.
System matrices are G0: 
 [[0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.99936047 0.         0.         0.        ]] and 
 F0:
[[0.08675441 0.         0.         0.         0.        ]
 [0.         0.29949582 0.         0.         0.        ]].
System matrices are G1: 
 [[0.         0.         0.00105992 0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.00105992 0.         0.        ]] and 
 F1:
[[0.         0.         0.00379384 0.         0.28345207]
 [0.         0.         0.         0.         0.        ]].

SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import logging
import pickle
import signal
import sys
import numpy as np

def setup_logging(name):
    """配置日志记录"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f'{name}_run.log'),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(name)

def save_params(params, filename):
    """保存参数到文件"""
    with open(filename, 'wb') as f:
        pickle.dump(params, f)
    logging.info(f"参数已保存到 {filename}")

def load_params(filename):
    """从文件加载参数"""
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return None

def handle_interrupt(signum, frame):
    """处理中断信号"""
    logging.warning(f"接收到中断信号 {signum}")
    sys.exit(1)

if __name__ == '__main__':
    # 初始化参数
    params = {
        'S_len': 10, 'I_len': 16, 'T_len': 20, 'F_len': 2,
        'J': 5, 'name': "lds", 'hiddenstates': [5, 2],
        'windows': [3], 'feature_selection': [2],
        'reg': np.array([270]), 'methods': ['IF-Gurobi','IF','EM','FFT','DTW'],
        'T_start': 5, 'T_rate': 5, 'thresh': 0.25
    }

    # 设置日志和中断处理
    logger = setup_logging(params['name'])
    signal.signal(signal.SIGINT, handle_interrupt)
    signal.signal(signal.SIGTERM, handle_interrupt)

    # 保存参数
    param_file = f"{params['name']}_params.pkl"
    save_params(params, param_file)

    try:
        # 生成数据
        logger.info("开始生成数据...")
        data, label = utils_MIP4Cluster().datacleaning([], params['name'],
                                                     params['S_len'], params['I_len'],
                                                     params['T_len'], params['F_len'],
                                                     params['J'])

        # 准备参数
        window = np.array([[w] for w in params['windows']])
        F = np.array([[f] for f in params['feature_selection']])
        N = np.array([[n] for n in params['hiddenstates']])

        # 运行测试
        for i in range(data.shape[0]):
            num = 2 + i
            data_X = data[i]
            label_X = label[i]
            logger.info(f"开始处理数据集 {num}...")

            MIP4Cluster().Test_MIP4Cluster(
                data_X, label_X, params['methods'],
                params['S_len'], params['I_len'],
                window, F, f"{params['name']}{num}",
                N, params['J'], params['T_start'],
                params['T_rate'], params['reg'],
                thresh=params['thresh']
            )

            logger.info(f"数据集 {num} 处理完成")

        logger.info("所有处理完成！")

    except Exception as e:
        logger.error(f"处理失败: {str(e)}", exc_info=True)
    finally:
        logger.info("程序结束")

Final data shape: (3, 10, 32, 20, 2)
Labels shape: (3, 10, 32)
(10, 32, 20, 2) (10, 32)
(1, 1) (2, 1)
(21, 3, 2) (21,)
m,t: 2 3
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2611849
Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Set parameter TimeLimit to value 360
5 5 2 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  360
NonConvex  2

Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Optimize a model with 4 rows, 357 columns and 84 nonzeros
Model fingerprint: 0x4e307d47
Model has 196 quadratic objective terms
Model has 196 quadratic constraints
Variable types: 336 continuous, 21 integer (21 binary)
Coefficient statistics:


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from IPython.display import display, Markdown

def display_log(log_file="lds_run.log"):
    try:
        with open(log_file, 'r') as f:
            display(Markdown(f"```\n{f.read()}\n```"))
    except FileNotFoundError:
        print("日志文件不存在")

display_log()

```
2025-04-02 13:10:44,143 - INFO - 开始生成数据...
2025-04-02 13:10:44,172 - INFO - 开始处理数据集 2...
2025-04-02 13:11:39,220 - WARNING - 接收到中断信号 2
2025-04-02 13:11:39,223 - INFO - 程序结束
2025-04-02 13:12:33,431 - INFO - 开始生成数据...
2025-04-02 13:12:33,431 - INFO - 开始生成数据...
2025-04-02 13:12:33,483 - INFO - 开始处理数据集 2...
2025-04-02 13:12:33,483 - INFO - 开始处理数据集 2...
2025-04-02 13:13:33,226 - WARNING - 接收到中断信号 2
2025-04-02 13:13:33,226 - WARNING - 接收到中断信号 2
2025-04-02 13:13:33,234 - INFO - 程序结束
2025-04-02 13:13:33,234 - INFO - 程序结束

```

In [ ]:
import pickle

# 定义文件名（根据实际保存的名称修改）
param_file = "lds_params.pkl"

# 读取参数
with open(param_file, "rb") as f:
    params = pickle.load(f)

# 打印参数内容
print("加载的参数：")
for key, value in params.items():
    print(f"{key}: {value}")

加载的参数：
S_len: 10
I_len: 16
T_len: 20
F_len: 2
J: 5
name: lds
hiddenstates: [5, 2]
windows: [3]
feature_selection: [2]
reg: [270]
methods: ['IF-Gurobi', 'IF', 'EM', 'FFT', 'DTW']
T_start: 5
T_rate: 5
thresh: 0.25


### Test

In [ ]:
# Don't know where to start, try here👇🏻:
if __name__ == '__main__':
    with open('notebook_output.txt', 'w', encoding='utf-8') as f:
    # Self-setting
    S_len, I_len, T_len, F_len =10, 16, 20, 2
    J = 5
    name = "lds"
    hiddenstates =[5,2]
    windows = [3]
    feature_selection = [2]
    # regularization set
    # method= ['IF-Gurobi']
    # reg = np.array(range(20, 100, 50))
    reg = np.array([270])

    # Generate Data
    # data_dir=utils_MIP4Cluster().data_generation(g,f_dash,pro_rang,obs_rang,T_len,S_len)
    data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
    method= ['IF-Gurobi','IF','EM','FFT','DTW']
    window = np.array([[w] for w in windows])# Timeset
    F = np.array([[f] for f in feature_selection])# Feature
    N = np.array([[n] for n in hiddenstates])
    T_start = 5
    T_rate = 5
            self.terminal.write(message)
        self.log.write(message)
    for i in range(data.shape[0]):
        num =2+i
        data_X = data[i]
        label_X = label[i]
        MIP4Cluster().Test_MIP4Cluster(data_X, label_X, method, S_len, I_len, window, F, f"lds{num}", N, J, T_start, T_rate, reg, thresh=0.25)

**Reconstruct Data**

We generated data with hidden states 2,3,4 as './data/2_2_test.npy', './data/3_2_test.npy', './data/4_2_test.npy'. Now we have test results 3*3*50 for testing generated data with hidden states 2,3,4 seperately. We extract the diagonal elements for the testing results of testing data in 2D dimensions 3*50.

In [ ]:
import numpy as np

if not os.path.exists(f'./Result_{name}'):
    os.makedirs(f'./Result_{name}')
name='lds'

for method in ['IF-Gurobi','EM','IF','FFT','DTW']:
    arr_f1_=[]
    arr_mean_ =[]
    arr_std_=[]
    arr_dura_=[]
    result_f1=[[],[],[]]
    result_mean=[[],[],[]]
    result_std=[[],[],[]]
    result_dura=[[],[],[]]
    for i in range(3):
        result_f1[i] = np.load(f'./Result_{name}{i+2}/f1_{method}_{name}{i+2}.npy')  # 形状应为 (3, 50)
        result_mean[i] = np.load(f'./Result_{name}{i+2}/f1_{method}_{name}{i+2}_mean.npy')  # 形状应为 (3, 1)
        result_std[i] = np.load(f'./Result_{name}{i+2}/f1_{method}_{name}{i+2}_std.npy')  # 形状应为 (3, 1)
        result_dura[i] = np.load(f'./Result_{name}{i+2}/duration_{method}_{name}{i+2}.npy')  # 形状应为 (3, 1)
        #print(result_f1[i])
        arr_f1 = result_f1[i][i]
        arr_mean = result_mean[i][i]
        arr_std = result_std[i][i]
        arr_dura = result_dura[0][i]
        arr_f1_.append(arr_f1)
        arr_mean_.append(arr_mean)
        arr_std_.append(arr_std)
        arr_dura_.append(arr_dura)
    # 保存合并后的数组
    np.save(f'./Result_{name}/f1_{method}_{name}.npy', arr_f1_)
    np.save(f'./Result_{name}/f1_{method}_{name}_mean.npy', arr_mean_)
    np.save(f'./Result_{name}/f1_{method}_{name}_std.npy', arr_std_)
    np.save(f'./Result_{name}/duration_{method}_{name}.npy', arr_dura_)

    print("合并后的数组形状:", np.array(arr_f1_))  # 应该是 (3, 50)
    print("合并后的数组形状:", np.array(arr_mean_).shape)  # 应该是 (3, 1)
    print("合并后的数组形状:", np.array(arr_std_).shape)  # 应该是 (3, 1)
    print("合并后的数组形状:", np.array(arr_dura_).shape)  # 应该是 (3, 1)


### **Plots**

In [ ]:
method =['IF-Gurobi','EM','IF','FFT','DTW']

utils_MIP4Cluster().plot_MIF4cluster_methods( f'./Result_{name}/', method, name, cutdown = False )

## Test Real Data

### ECG Test

In [ ]:
# Test Real Data time window-2,3,4,5,6
if __name__ == '__main__':
    S_len, I_len, T_len, F_len =1, 15, 140, 1
    J = 2
    name = "ecg"
    hiddenstates =[2,3,4]
    windows = [2,3,4,5,6]
    feature_selection = [1]
    reg = np.array([270]) # regularization set

    # Generate Data
    # data_dir=utils_MIP4Cluster().data_generation(g,f_dash,pro_rang,obs_rang,T_len,S_len)
    data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
    method= ['IF-Gurobi','IF','EM','FFT','DTW']
    window = np.array([[w] for w in windows])# Timeset
    F = np.array([[f] for f in feature_selection])# Feature
    N = np.array([[n] for n in hiddenstates])
    T_start = 30
    T_rate = 70
    MIP4Cluster().Test_MIP4Cluster(data, label, method, S_len, I_len, window, F, name, N, J, T_start, T_rate, reg, thresh=0.25)

target
b'1'    292
b'2'    177
b'4'     19
b'3'     10
b'5'      2
Name: count, dtype: int64
[[[[-0.39452983]
   [-3.586584  ]
   [-4.2591798 ]
   ...
   [-0.38534423]
   [ 0.47653489]
   [-0.48646838]]

  [[ 0.1628937 ]
   [-1.6618931 ]
   [-2.6876828 ]
   ...
   [ 0.22211322]
   [ 0.93800701]
   [ 0.96397359]]

  [[-1.0759223 ]
   [-2.4474781 ]
   [-3.4529485 ]
   ...
   [-1.1122086 ]
   [-1.2669268 ]
   [-2.5715131 ]]

  ...

  [[ 1.0850975 ]
   [ 0.77107384]
   [ 0.06014576]
   ...
   [-4.3219719 ]
   [-4.874837  ]
   [-4.5535132 ]]

  [[-0.16309261]
   [-1.1107048 ]
   [-2.0518539 ]
   ...
   [-2.9481784 ]
   [-2.0135895 ]
   [-0.84451256]]

  [[-0.92282223]
   [-1.1388419 ]
   [-1.1923367 ]
   ...
   [-4.1909718 ]
   [-4.8443645 ]
   [-3.1053415 ]]]]
Final data shape: (1, 30, 140, 1)
Labels shape: (1, 30)
(1, 30, 140, 1) (1, 30)
(5, 1) (3, 1)
(20, 2, 1) (20,)
m,t: 1 2
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2611849
Academic license 26118

Getting files...
Done
-----------------------------------------------------------------
IDAES Extensions Build Versions
Solvers:  v3.4.2 20240811 ubuntu2204-x86_64
Library:  v3.4.2 20240811 ubuntu2204-x86_64



default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.11/dist-packages/pyomo/core/base/indexed_component.py:718)


(5, 1) (3, 1)
(20, 2, 1) (20,)
m,t: 1 2
Bonmin 1.8.8 using Cbc 2.10.10 and Ipopt 3.13.2
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a colle

default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.11/dist-packages/pyomo/core/base/indexed_component.py:718)
default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.11/dist-packages/pyomo/core/base/indexed_component.py:718)


(5, 1) (3, 1)
(20, 2, 1) (20,)
m,t: 1 2
12 8
0 start: idx_= [0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 0]
cluster length: 12 8
0 changed: idx= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
K: 20 
 L: 0
1 start: idx_= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
cluster length: 20 0
1 changed: idx= [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
K: 18 
 L: 2
f1-score: 0.7142857142857143
n:2.EM_F1:0.7142857142857143
(17, 2, 1) (17,)
m,t: 1 2
8 9
0 start: idx_= [0 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1]
cluster length: 8 9
0 changed: idx= [0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1]
K: 3 
 L: 14
1 start: idx_= [0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1]
cluster length: 3 14
1 changed: idx= [1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1]
K: 3 
 L: 14
f1-score: 0.5714285714285714
n:2.EM_F1:0.5714285714285714
(20, 3, 1) (20,)
m,t: 1 3
12 8
0 start: idx_= [0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 1 0]
cluster length: 12 8
0 changed: idx= [1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0]
K: 10 
 L: 10
1 start: idx_= [1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0]


AttributeError: '_PyDrive2ImportHook' object has no attribute 'find_spec'

In [ ]:
if max(max(T)) <10:
    cutdown =True
utils_MIP4Cluster.plot_MIF4cluster_methods( f'./Result_{name}/', method, name, cutdown=cutdown )

In [ ]:
# Test Real Data time window-30,60,90,120,140
if __name__ == '__main__':
    name = "ecg"
    method= ['IF','EM','FFT','DTW']
    S_len, I_len, T_len, M_len =10, 15, 140, 1
    data, label, S, I, T, M = utils_MIP4Cluster.datacleaning(name, S_len, I_len, T_len, M_len)
    cutdown = False
    S=50
    reg = [270]
    I=5
    T=[30,60,90,120,140]
    N = [2, 3, 4]
    M=1
    MIP4Cluster().Test_MIP4Cluster(data, label, method, S, I, T, M, reg, name, N)

(10, 16, 20, 2) (10, 16, 20, 2)
(10, 16, 20, 2) (10, 16, 20, 2)
(10, 16, 20, 2) (10, 16, 20, 2)
(10, 16, 20, 2) (10, 16, 20, 2)
(10, 16, 20, 2) (10, 16, 20, 2)
(10, 16, 20, 2) (10, 16, 20, 2)


UnboundLocalError: cannot access local variable 'os' where it is not associated with a value

###Scalp EEG Test

We use CHB-MIT Scalp electroencephalogram (EEG) Database to cluster seizures in epileptic and neurotypical patients. This work contains the code used for Erin Shappell's Honors Thesis submitted in Spring 2020.

In particular we draw from the EEG datasets:

[CHB-MIT Scalp EEG Database](https://archive.physionet.org/pn6/chbmit/)

------
*  **eeg_seizure.npy(n_patients, n_samples, n_timesteps, n_channels)**: test data with dimensions 3, 14(7 seizure and 7 normal), 12800, 3

------
The CHB-MIT dataset contains data from many inviduals, but we will use only the 1,3,5. Each individual was chose 14(7 seizure and 7 normal) recordings saved as EDF files, and each file has 3 channels associated with different areas of the brain. Additionally, each EDF file either contains or doesn't contain a single seizure event. For more information, see

[Data Clearning-Colab](https://colab.research.google.com/drive/1xrklpGNdiyrKlJRdS9-lP5DQRH2FiCQT#scrollTo=3vZWQC8FmLlK)
[Github link](https://github.com/sereneHe/MIP-IF-4JointProblem/blob/main/Database/EEG/eeg_seizure_prediction.ipynb)


In [ ]:
# Don't know where to start, try here👇🏻:
if __name__ == '__main__':
    # Self-setting
    S_len, I_len, T_len, F_len =3, 7, 12800, 3
    J = 10
    name = "seizure-eeg"
    hiddenstates =[3,4]
    windows = [10]
    feature_selection = [3]
    reg = np.array([270])

    data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
    method= ['IF-Gurobi','IF','EM','FFT','DTW']
    window = np.array([[w] for w in windows])# Timeset
    F = np.array([[f] for f in feature_selection])# Feature
    N = np.array([[n] for n in hiddenstates])
    T_start=10240
    T_rate=265
    MIP4Cluster().Test_MIP4Cluster(data, label, method, S_len, I_len, window, F, name, N, J, T_start,T_rate, reg, thresh=0.25)

Final data shape: (3, 14, 12800, 3)
Labels shape: (3, 14)
(3, 14, 12800, 3) (3, 14)
(1, 1) (2, 1)
(11, 10, 3) (11,)
m,t: 3 10
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2611849
Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Set parameter TimeLimit to value 360
3 3 9 3
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  360
NonConvex  2

Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Optimize a model with 4 rows, 977 columns and 324 nonzeros
Model fingerprint: 0xf3e13375
Model has 702 quadratic objective terms
Model has 702 quadratic constraints
Variable types: 966 continuous, 11 integer (11 binary)
Coefficient statistics:
 

GurobiError: Unable to retrieve attribute 'x'

### Codiet Data-biosensors

[Codiet](https://azti.sharepoint.com/sites/Codiet/Documentos%20compartidos/Forms/AllItems.aspx?csf=1&web=1&e=Ix5HZs&CID=4001f6d7%2Dfb38%2D4f5f%2Dbd80%2Deb7c0a08324b&FolderCTID=0x012000471C64E07A2B974B84495CF678C296F1&id=%2Fsites%2FCodiet%2FDocumentos%20compartidos%2FGeneral%2F05%20Work%20Packages%2FWP2%5FBIOMARKER%20PLUS%2FWP2%20pre%2Dprocessed%20data%2FCleaned%20data%2Fcleaned&viewid=89cbb0d8%2D0a9f%2D4e1e%2Dbc77%2D7c6891c784b9) data can be downloaded online. Upload .pkl files into the Python data frame directly. Visit V1 prior to the 8-week period and V3 after.


In [ ]:
# Don't know where to start, try here👇🏻:
if __name__ == '__main__':
    # Self-setting
    S_len, I_len, T_len, F_len =6, 23, 3, 3
    J = 5
    name = "biosensors"
    hiddenstates =[2,3,4]
    windows = [3]
    feature_selection = [2]
    reg = np.array([270])
    data, label = utils_MIP4Cluster().datacleaning([], name, S_len, I_len, T_len, F_len, J)
    method= ['FFT','DTW', 'IF-Gurobi','IF','EM']
    window = np.array([[w] for w in windows])# Timeset
    F = np.array([[f] for f in feature_selection])# Feature
    N = np.array([[n] for n in hiddenstates])
    MIP4Cluster().Test_MIP4Cluster(data, label, method, S_len, I_len, window, F, name, N, J, reg, thresh=0.75)

(6, 23, 3, 3) (6, 23)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.6 MB/s eta 0:00:00
(1, 1) (3, 1)
[1 1 0 1 1 0 1 1 1 1 1]
Initialization complete
Iteration 0, inertia 4402.8011392846.
Iteration 1, inertia 1698.9586344367424.
Converged at iteration 1: strict convergence.
[0 0 1 0 0 1 1 0 0 0 0]
0.9411764705882353
FFT_F1:0.9411764705882353
[1 1 1 1 1 0 1 0 1 1 1]
[0 0 0 0 0 1 0 1 0 1 1]
0.875
FFT_F1:0.875
[1 1 0 1 1 1 0 1 1 1 1]
[1 1 0 1 1 1 0 0 1 1 1]
0.9411764705882353
FFT_F1:0.9411764705882353
[1 1 1 1 1 0 0 1 1 1 1]
[0 0 0 1 0 1 1 0 0 1 0]
0.875
FFT_F1:0.875
[1 1 1 1 1 1 0 1 1 1 0]
[0 0 0 0 0 1 1 0 0 1 1]
0.875
FFT_F1:0.875
[0 1 0 0 1 0 1 0 0 1]
Initialization complete
Iteration 0, inertia 4345.676513676276.
Iteration 1, inertia 3072.7685271063583.
Converged at iteration 1: strict convergence.
[1 0 0 0 0 1 0 0 1 1]
0.6
FFT_F1:0.6
[0 1 1 0 0 0 0 1 1 0]
[1 1 1 0 1 0 1 1 0 1]
0.5454545454545454
FFT_F1:0.5454545454545454
[0 0 0 1 0 0 0 1 1 1]
[0 1 0 1 1 1 1 0 1 1]
0.545

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


34.995 --> 11.273 --> 11.295 --> 11.288 --> 11.289 --> 11.289 --> 
[1 1 0 0 0 0 1 0 0 1 1]
f1: 0.7142857142857143
DTW_F1:0.7142857142857143
[1 1 1 1 1 0 1 0 1 1 1]
3
Soft-DTW k-means
32.195 --> 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


10.895 --> 10.896 --> 10.896 --> 
[1 0 0 1 1 0 1 0 1 1 1]
f1: 0.875
DTW_F1:0.875
[1 1 0 1 1 1 0 1 1 1 1]
3
Soft-DTW k-means


...

f1-score: 0.7272727272727273
n:4,IF_F1:0.7272727272727273.
m,t: 2 3
Bonmin 1.8.8 using Cbc 2.10.10 and Ipopt 3.13.2
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowled

  - termination condition: infeasible
  - message from solver: bonmin\x3a Infeasible problem


f1-score: 0.5454545454545454
n:4,IF_F1:0.5454545454545454.
m,t: 2 3
Bonmin 1.8.8 using Cbc 2.10.10 and Ipopt 3.13.2
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowled

  - termination condition: infeasible
  - message from solver: bonmin\x3a Infeasible problem


f1-score: 0.6
n:4,IF_F1:0.6.
f1_IF_biosensors_mean is: 
 [0.65626022 0.65626022 0.65626022], f1_IF_biosensors_std is: 
 [0.17475044 0.17475044 0.17475044], duration_IF_biosensors is: 
 [135.39377475 537.56415892 847.74740124]. 
Getting files...
Done
-----------------------------------------------------------------
IDAES Extensions Build Versions
Solvers:  v3.4.2 20240811 ubuntu2204-x86_64
Library:  v3.4.2 20240811 ubuntu2204-x86_64

(1, 1) (3, 1)
m,t: 2 3
3 8
0 start: idx_= [1 1 1 1 1 1 1 1 0 0 0]
cluster length: 3 8
0 changed: idx= [1 1 0 1 1 1 1 0 0 0 0]
K: 5 
 L: 6
1 start: idx_= [1 1 0 1 1 1 1 0 0 0 0]
cluster length: 5 6
1 changed: idx= [1 1 0 1 1 1 0 0 1 0 0]
K: 5 
 L: 6
2 start: idx_= [1 1 0 1 1 1 0 0 1 0 0]
cluster length: 5 6
2 changed: idx= [1 1 0 1 1 1 0 0 1 0 1]
K: 4 
 L: 7
f1-score: 0.75
n:2.EM_F1:0.75
m,t: 2 3
8 3
0 start: idx_= [0 0 0 0 0 0 0 0 1 1 1]
cluster length: 8 3
0 changed: idx= [0 0 1 0 0 0 0 1 1 1 1]
K: 6 
 L: 5
1 start: idx_= [0 0 1 0 0 0 0 1 1 1 1]
cluster le

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score

df = pd.read_csv('./data/biosensors.csv')
target_columns = ['patient','risk-group', 'bmi-(body-mass-index)', 'pbf-(percent-body-fat)', 'vfl-(visceral-fat-level)','site','gender','visit','date-of-birth','exam-date']
df = df[target_columns].dropna(subset=target_columns).reset_index(drop=True)
df.columns = ['patient','risk-group','BMI', 'PBF', 'VFL','site','gender','visit','date-of-birth','exam-date']
risk_mapping = {'Normal': 0, 'I': 1, 'II': 2}
df['risk-group-numeric'] = df['risk-group'].map(risk_mapping)
df['VFL'] = df['VFL'].str.extract('(\d+)').astype(float)
#print(df)

# Train data
patients = df['patient'].unique()
num_patients = len(patients)
time_steps = 3  # 每个患者有3次随访
num_features = 3  # 选择3个特征：BMI, PBF, VFL

X_data = []
for i, patient in enumerate(patients):
    patient_data = df[df['patient'] == patient]#.sort_values('visit')
    #print(patient_data)
    if patient_data.shape[0] ==3:
        X_data.append(patient_data[['BMI','PBF','VFL']].values)
print(X_data)

# Label
risk_sum = (df.groupby('patient')
           .filter(lambda x: len(x) == 3)  # 只保留有≥3次随访的患者
           .groupby('patient')['risk-group-numeric']
           .sum()
           .reset_index())
#print(len(risk_sum))
risk_sum.columns = ['patient', 'total_risk_score']
risk_sum['label'] = (risk_sum['total_risk_score'] >= 3).astype(int)
label_data = risk_sum['label'].values
print(label_data)
X=np.array(X_data).reshape(6, 23, 3, 3)
label=np.array(label_data).reshape(6, 23)
print(X[:2],label)

[array([[29.5, 38.4, 16. ],
       [30. , 36.7, 15. ],
       [29.6, 35.5, 14. ]]), array([[40.5, 49.1, 28. ],
       [40.2, 47.9, 28. ],
       [39.5, 46.7, 27. ]]), array([[26.2, 28.8, 11. ],
       [25.9, 30. , 11. ],
       [26. , 28.2, 11. ]]), array([[45. , 47.9, 30. ],
       [45.5, 46.9, 29. ],
       [45.6, 46.5, 30. ]]), array([[31.9, 48.2, 21. ],
       [32.8, 46.3, 21. ],
       [32.6, 47.8, 22. ]]), array([[34.9, 38. , 22. ],
       [35. , 38.3, 22. ],
       [34.4, 38.5, 22. ]]), array([[30.6, 33.6, 13. ],
       [29.4, 33.2, 12. ],
       [29.4, 33.5, 12. ]]), array([[29.3, 28.6, 12. ],
       [29.6, 29.6, 13. ],
       [29.6, 28.3, 13. ]]), array([[35.7, 39.6, 21. ],
       [35.5, 39.1, 21. ],
       [34.9, 39.5, 21. ]]), array([[27.2, 37.8, 14. ],
       [26.4, 38.8, 14. ],
       [26.9, 35.6, 13. ]]), array([[30.1, 33. , 12. ],
       [29.6, 32.5, 11. ],
       [29.8, 33.3, 11. ]]), array([[34.7, 34.1, 19. ],
       [34. , 33.7, 19. ],
       [34.3, 34.4, 19. ]]), arr

### **Prevalence of Depression**
Among U.S. Adults (2013-2016)
## **Background**
This R code replicates **NCHS Data Brief No. 303**, analyzing the **prevalence of depression among U.S. adults aged 20 and over** from **2013 to 2016**.

### **Data Source**
- **National Health and Nutrition Examination Survey (NHANES)**  
- **Depression Screener Questionnaire (DPQ)**  
- **Demographic Data (DEMO)**  

### **Study Objectives**
- **Estimate the overall prevalence of depression**
- **Compare depression rates by sex and age group**
- **Perform statistical tests for differences between groups**

---

## **1. Data Structure**
### **Key Datasets**
| Dataset | Description |
|---------|------------|
| `DEMO_H` & `DEMO_I` | Demographic information (age, sex, survey weights, etc.) |
| `DPQ_H` & `DPQ_I` | Depression screener questionnaire (PHQ-9 scores) |

### **Key Variables**
| Variable | Description |
|----------|------------|
| `SEQN` | Participant ID |
| `RIAGENDR` | Sex (1 = Male, 2 = Female) |
| `RIDAGEYR` | Age (in years) |
| `SDMVSTRA` | Statistical strata |
| `SDMVPSU` | Primary sampling unit (PSU) |
| `WTMEC2YR` | 2-year survey weight |
| `DPQ010 - DPQ090` | 9 depression-related questions (PHQ-9) |
| `Depression.Score` | Total PHQ-9 depression score |
| `Depression` | Depression indicator (PHQ-9 ≥ 10 = 100, else 0) |
| `Age.Group` | Age groups (20-39, 40-59, 60+) |
                   
                                            

In [ ]:
#' ------------------------------------------------------------------------------------------------------------
#' Example R code to replicate NCHS Data Brief No.303, Figures 1
#' Prevalence of Depression Among Adults Aged 20 and Over: United States, 2013-2016

#' Brody DJ, Pratt LA, Hughes JP. Prevalence of Depression Among Adults Aged 20 and Over: United
#' States, 2013-2016. NCHS Data Brief. No 303. Hyattsville, MD: National Center for Health Statistics. 2018.

#' Available at: https://www.cdc.gov/nchs/products/databriefs/db303.htm

#' ------------------------------------------------------------------------------------------------------------

# Load survey and dplyr packages
#+ message = FALSE, warning=FALSE
library(dplyr)
library(survey)
#'
options(survey.lonely.psu='adjust')

# Display Version Information
cat("R package versions:\n")
for (p in c("base", "survey","dplyr")) {
  cat(p, ": ", as.character(packageVersion(p)), "\n")
}

#' # Data preparation
# Download & Read SAS Transport Files
# Demographic (DEMO)
download.file("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/DEMO_H.XPT", tf <- tempfile(), mode="wb")
DEMO_H <- foreign::read.xport(tf)[,c("SEQN","RIAGENDR","RIDAGEYR","SDMVSTRA","SDMVPSU","WTMEC2YR")]
download.file("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2015/DataFiles/DEMO_I.xpt", tf <- tempfile(), mode="wb")
DEMO_I <- foreign::read.xport(tf)[,c("SEQN","RIAGENDR","RIDAGEYR","SDMVSTRA","SDMVPSU","WTMEC2YR")]

# Mental Health - Depression Screener (DPQ)
download.file("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/DPQ_H.xpt", tf <- tempfile(), mode="wb")
DPQ_H <- foreign::read.xport(tf)
download.file("https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2015/DataFiles/DPQ_I.xpt", tf <- tempfile(), mode="wb")
DPQ_I <- foreign::read.xport(tf)

# Append Files
DEMO <- bind_rows(DEMO_H, DEMO_I)
DPQ <- bind_rows(DPQ_H, DPQ_I)

# Merge DEMO and DPQ files and create derived variables

One <- left_join(DEMO, DPQ, by="SEQN") %>%
  # Set 7=Refused and 9=Don't Know To Missing for variables DPQ010 thru DPQ090 ##
  mutate_at(vars(DPQ010:DPQ090), ~ifelse(. >=7, NA, .)) %>%
  mutate(. ,
         # create indicator for overall summary
         one = 1,
         # Create depression score as sum of variables DPQ010 -- DPQ090
         Depression.Score = rowSums(select(. , DPQ010:DPQ090)),
         # Create depression indicator as binary 0/100 variable. (is missing if Depression.Score is missing)
         Depression= ifelse(Depression.Score >=10, 100, 0),
         # Create factor variables
         Sex = factor(RIAGENDR, labels=c("Men", "Women")),
         Age.Group = cut(RIDAGEYR, breaks=c(-Inf,19,39,59,Inf),labels=c("Under 20", "20-39","40-59","60 and over")),
         # Generate 4-year MEC weight (Divide weight by 2 because we are appending 2 survey cycles)
         # Note: using the MEC Exam Weights (WTMEC2YR), per the analytic notes on the
         #       Mental Health - Depression Screener (DPQ_H) documentation
         WTMEC4YR = WTMEC2YR/2 ,
         # Define indicator for analysis population of interest: adults aged 20 and over with a valid depression score
         inAnalysis= (RIDAGEYR >= 20 & !is.na(Depression.Score))
  ) %>%
  # drop DPQ variables
  select(., -starts_with("DPQ"))

#' ## Define survey design
# Define survey design for overall dataset
NHANES_all <- svydesign(data=One, id=~SDMVPSU, strata=~SDMVSTRA, weights=~WTMEC4YR, nest=TRUE)

# Create a survey design object for the subset of interest: adults aged 20 and over with a valid depression score
# Subsetting the original survey design object ensures we keep the design information about the number of clusters and strata
NHANES <- subset(NHANES_all, inAnalysis)

#' ## Analysis
# Define a function to call svymean and unweighted count
getSummary <- function(varformula, byformula, design){
  # Get mean, stderr, and unweighted sample size
  c <- svyby(varformula, byformula, design, unwtd.count )
  p <- svyby(varformula, byformula, design, svymean )
  outSum <- left_join(select(c,-se), p)
  outSum
}

#' ### Calculate prevalence of depression overall, by sex, by age group, and by age and sex
#' Adults
getSummary(~Depression, ~one, NHANES)
#' By sex
getSummary(~Depression, ~Sex, NHANES)
#' By age
getSummary(~Depression, ~Age.Group, NHANES)
#' By sex and age
getSummary(~Depression, ~Sex + Age.Group, NHANES)

#' ### Compare Prevalence Between Men And Women
svyttest(Depression~Sex, NHANES)$p.value %>% as.numeric
svyttest(Depression~Sex, subset(NHANES, Age.Group=="20-39"))$p.value %>% as.numeric
svyttest(Depression~Sex, subset(NHANES, Age.Group=="40-59"))$p.value %>% as.numeric
svyttest(Depression~Sex, subset(NHANES, Age.Group=="60 and over"))$p.value %>% as.numeric


#' ### Pairwise t-testing by age groups for total, men, and women
#' Differences by age group, among all adults
# Full output from svyttest command
svyttest(Depression~Age.Group, subset(NHANES, Age.Group=="20-39" | Age.Group=="40-59"))
# Displaying p-values only
svyttest(Depression~Age.Group, subset(NHANES, Age.Group=="20-39" | Age.Group=="40-59"))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Age.Group=="20-39" | Age.Group=="60 and over"))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Age.Group=="40-59" | Age.Group=="60 and over"))$p.value %>% as.numeric
#' Differences by age group, among men
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Men" & (Age.Group=="20-39" | Age.Group=="40-59")))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Men" & (Age.Group=="20-39" | Age.Group=="60 and over")))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Men" & (Age.Group=="40-59" | Age.Group=="60 and over")))$p.value %>% as.numeric
#' Differences by age group, among women
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Women" & (Age.Group=="20-39" | Age.Group=="40-59")))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Women" & (Age.Group=="20-39" | Age.Group=="60 and over")))$p.value %>% as.numeric
svyttest(Depression~Age.Group, subset(NHANES, Sex=="Women" & (Age.Group=="40-59" | Age.Group=="60 and over")))$p.value %>% as.numeric



In [ ]:
# Test You Data? Dont know the dimention? No worries!
if __name__ == '__main__':
    # time
    S_len, I_len, T_len, M_len =[], [], [], []
    name = "You_Data_Name(Connect with '_' for more than one word)"
    data, label, S, I, T, M = utils_MIP4Cluster.datacleaning(name, S_len, I_len, T_len, M_len)
    method= ['IF-Gurobi','IF','EM','FFT','DTW']
    reg = [270]
    I=5
    T=2
    N = [2, 3, 4]
    M=1
    MIP4Cluster().Test_MIP4Cluster(data, label, method, S, I, T, M, reg, name, N)